In [1]:
import pandas as pd
import numpy as np


def calculate_distancia_path(nodo_seleccionado, df_rutas, ruta = [], distancia_acumulada = 0):
    ## criterio de parada
    if len(ruta)==8:
        return distancia_acumulada, ruta
    
    else:
        p_asociados = df_rutas.loc[nodo_seleccionado]
        ruta_minima = []
        
        distancia_acumulada_minima = 10**5
        for rows in p_asociados.iterrows():
            index = rows[0]
            fila = rows[1]
            
            ## -------------------------
            distancia = fila[0]
            next_point = fila[2]
            
            if next_point not in ruta:
                ruta_list = ruta + [next_point]
                distancia_acumulada_list = distancia_acumulada + distancia
                distancia_acumulada_path, ruta_path = calculate_distancia_path(next_point, df_rutas, ruta_list, distancia_acumulada_list)
                
                ## Selecciona las distncias minimas para caminos más pequeños
                if distancia_acumulada_path <= distancia_acumulada_minima:
                    distancia_acumulada_minima = distancia_acumulada_path 
                    ruta_minima = ruta_path
        return distancia_acumulada_minima, ruta_minima 
        

def shortest_path(p_iniciales, df_rutas):
    distancia_acumulada_minima = 10**10
    for partida in p_iniciales:
        ruta = [partida]
        distancia_acumulada, ruta_solution = calculate_distancia_path(partida, df_rutas, ruta)
        
        if distancia_acumulada < distancia_acumulada_minima:
            distancia_acumulada_minima = distancia_acumulada 
            ruta_minima = ruta_solution 

    return  distancia_acumulada_minima, ruta_minima


In [2]:


### Lectura de archivo rutas.txt
print("reading file rutas.txt")
print("#"*10)
df_rutas = pd.read_table("rutas.txt", header=None)
df_rutas.columns = ["rutas_description"]
lista_splits = df_rutas["rutas_description"].apply(lambda x: x.split("to"))
df_rutas["partida"] = lista_splits.apply(lambda x: x[0].strip())
df_rutas["llegada"] = lista_splits.apply(lambda x: x[1].split("=")[0].strip()) 
df_rutas["distancia"] =  lista_splits.apply(lambda x: x[1].split("=")[1].strip()) 
del df_rutas["rutas_description"]

p_iniciales = set(list(df_rutas.partida.unique())+list(df_rutas.llegada.unique()))

lista_df = []
for punto in p_iniciales:
    if df_rutas[df_rutas.partida==punto].shape[0]<6:
        df_add = df_rutas[df_rutas.llegada==punto].copy()
        df_add.columns = ["llegada", "partida", "distancia"]
        cols =  ["partida","llegada","distancia"]
        lista_df.append(df_add[cols])
        
df_rutas_complete = pd.concat([df_rutas]+lista_df).drop_duplicates().reset_index()
df_rutas_2 = df_rutas_complete[["distancia","partida","llegada"]].set_index(["partida","llegada"], drop =False)
df_rutas_2["distancia"] = df_rutas_2["distancia"].astype(int)
p_iniciales = set(list(df_rutas.partida.unique())+list(df_rutas.llegada.unique()))

## Calculo de camino más corto
print("Calculando")
print("#"*10)
distancia_acumulada, ruta_minima = shortest_path(p_iniciales, df_rutas_2)
print((distancia_acumulada, ruta_minima))

reading file rutas.txt
##########
Calculando
##########
(141, ['Tristram', 'AlphaCentauri', 'Norrath', 'Straylight', 'Faerun', 'Snowdin', 'Tambi', 'Arbre'])
